In [38]:
#imports
import pandas as pd
import torch
from collections import Counter
from transformers import AutoModel, AutoTokenizer
import unicodedata
from datasets import load_dataset
import itertools

import os
import re
import numpy as np 
from sklearn.metrics import accuracy_score

import transformers

import torch
from torch import cuda
from tqdm import tqdm_notebook as tqdm
device = 'cuda' if cuda.is_available() else 'cpu'

device

'cpu'

# The Data

Converting test, train and val datasets which include kanji to be purely kana (furigana)

In [2]:
#data_files = {"train": "train_split.csv", "test": "test_split.csv", 'val': 'val_split.csv'}
dataset = load_dataset('universal_dependencies', 'ja_gsd')
train_split = dataset['train']
test_split = dataset['test']
val_split = dataset['validation']

train_split_df = pd.DataFrame(data=train_split, columns=train_split.features)
train_split_df = train_split_df.drop(['lemmas','upos','xpos','feats','head','deprel','deps','misc'], axis=1)
train_split_df

test_split_df = pd.DataFrame(data=test_split, columns=test_split.features)
test_split_df = test_split_df.drop(['lemmas','upos','xpos','feats','head','deprel','deps','misc'], axis=1)
test_split_df

val_split_df = pd.DataFrame(data=val_split, columns=val_split.features)
val_split_df = val_split_df.drop(['lemmas','upos','xpos','feats','head','deprel','deps','misc'], axis=1)
val_split_df

Found cached dataset universal_dependencies (C:/Users/wlaw5/.cache/huggingface/datasets/universal_dependencies/ja_gsd/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)


  0%|          | 0/3 [00:00<?, ?it/s]

,idx,text,tokens
0,dev-s1,ただし、50周年ソングに変更後は、EDも歌つきのものが使われた。,"[ただし, 、, 50, 周年, ソング, に, 変更, 後, は, 、, ED, も, 歌..."
1,dev-s2,私は初めてだったんだけど思っていたよりも魚は新鮮でした。,"[私, は, 初めて, だっ, た, ん, だ, けど, 思っ, て, い, た, より, ..."
2,dev-s4,セントラル・リーグ審判員の水落朋大は実兄。,"[セントラル, ・, リーグ, 審判, 員, の, 水落, 朋大, は, 実兄, 。]"
3,dev-s5,多彩なライブアクトとともに、祝日前の渋谷の夜を盛り上げてくれそうだ。,"[多彩, な, ライブ, アクト, と, とも, に, 、, 祝日, 前, の, 渋谷, の..."
4,dev-s6,今作品では大会でのパラメータUPが非常に大きく、ALL999への育成は回復アイテムを投与しつ...,"[今, 作品, で, は, 大会, で, の, パラメータ, UP, が, 非常, に, 大..."
...,...,...,...
496,dev-s507,お風呂大きくはないけれどお湯はきれい。,"[お, 風呂, 大きく, は, ない, けれど, お, 湯, は, きれい, 。]"
497,dev-s508,新生闇軍団頭領。,"[新生, 闇, 軍団, 頭領, 。]"
498,dev-s509,"当初の説明と実際の施設の状態が違うという点も,川島町の住民の証言とよく似ています。","[当初, の, 説明, と, 実際, の, 施設, の, 状態, が, 違う, と, いう,..."
499,dev-s510,それでいてこの対応である。,"[それ, で, い, て, この, 対応, で, ある, 。]"


In [3]:
print(len(train_split))
print(len(test_split))
print(len(val_split))
#train_split['text']

7027
543
501


In [4]:
#load kanji data
kanji_df = pd.read_json("kanji.json")
kanji_df = kanji_df.drop(['strokes', 'grade', 'freq', 'jlpt_old', 'jlpt_new', 'meanings', 'wk_radicals', 'wk_readings_kun', 'wk_readings_on', 'wk_meanings', 'wk_level'], axis=0)

In [5]:
# As part of data collection, we need a dataset that is ONLY kana. 
# We could not find a dataset that had both kanji AND kana only version,
# so we took a dataset and converted the kanji to kana. 
# kanji to kana is less ambiguous with how on vs kun readings work, but there is still some abiguity.
# we recognize that here, in the dataset.
def unkanjify(df):
    new_sent = []
    for word in df['tokens']:
        #iterating list of chararacters
        for char in word:
            if char in kanji_df.columns:#is kanji
                if char in kanji_df.columns or char in kanji_df.columns:
                    if kanji_df[char].loc['readings_on']:
                        new_sent.append(kanji_df[char].loc['readings_on'][0])
                    else:
                        new_sent.append(char)
                #kun reading
                else:
                    for c in kanji_df[char].loc['readings_kun'][0]:
                        if c == '.':
                            continue
                        new_sent.append(c)
            else:#is not kanji
                new_sent.append(char)
    joined_sent = ''.join(new_sent)
    return joined_sent

# Applying the function to all splits (train, test and val)
train_split_df['kana'] = train_split_df.apply(lambda x: unkanjify(x), axis=1) 
test_split_df['kana'] = test_split_df.apply(lambda x: unkanjify(x), axis=1) 
val_split_df['kana'] = val_split_df.apply(lambda x: unkanjify(x), axis=1) 
train_split_df

,idx,text,tokens,kana
0,train-s1,ホッケーにはデンジャラスプレーの反則があるので、膝より上にボールを浮かすことは基本的に反則に...,"[ホッケー, に, は, デンジャラス, プレー, の, 反則, が, ある, の, で, ...",ホッケーにはデンジャラスプレーのはんそくがあるので、しつよりじょうにボールをふかすことはきほ...
1,train-s2,また行きたい、そんな気持ちにさせてくれるお店です。,"[また, 行き, たい, 、, そんな, 気持ち, に, さ, せ, て, くれる, お, ...",またこうきたい、そんなきじちにさせてくれるおてんです。
2,train-s3,手に持った特殊な刃物を使ったアクロバティックな体術や、揚羽と薄羽同様にクナイや忍具を使って攻...,"[手, に, 持っ, た, 特殊, な, 刃物, を, 使っ, た, アクロバティック, な...",しゅにじったとくしゅなじんぶつをしったアクロバティックなたいじゅつや、よううとはくうどうよう...
3,train-s4,"3年次にはトータルオフェンスで2,892ヤードを獲得し、これは大学記録となった。","[3, 年次, に, は, トータル, オフェンス, で, 2, ,, 892, ヤード, ...","3ねんじにはトータルオフェンスで2,892ヤードをかくとくし、これはだいがくきろくとなった。"
4,train-s5,葬儀の最中ですよ!,"[葬儀, の, 最中, です, よ, !]",そうぎのさいちゅうですよ!
...,...,...,...,...
7022,train-s7160,「さくらちゃん」と呼ばれている。,"[「, さくら, ちゃん, 」, と, 呼ば, れ, て, いる, 。]",「さくらちゃん」とこばれている。
7023,train-s7161,シェルマルケ氏は新憲法などをめぐり、アハメド暫定大統領と対立していた。,"[シェルマルケ, 氏, は, 新, 憲法, など, を, めぐり, 、, アハメド, 暫定,...",シェルマルケしはしんけんほうなどをめぐり、アハメドざんていだいとうりょうとたいりつしていた。
7024,train-s7162,"自らがオウム真理教ではない別のカルト団体に12年間所属していた経験をもとに,こう語りました。","[自ら, が, オウム, 真理, 教, で, は, ない, 別, の, カルト, 団体, に...",じらがオウムしんりきょうではないべつのカルトだんたいに12ねんかんしょぞくしていたけいけんを...
7025,train-s7163,紅い髪と同じ色の瞳という容姿に加え、眉と鎖骨の下の部分に二つずつ紅い球体が埋まっているという...,"[紅い, 髪, と, 同じ, 色, の, 瞳, と, いう, 容姿, に, 加え, 、, 眉...",こういはつとどうじしょくのどうというようしにかえ、びとさこつのかのぶぶんににつずつこういきゅ...


# download and intialize models

In [8]:
from transformers import BertTokenizer, TFBertForMaskedLM

tokenizer = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
model = TFBertForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


C:\Users\wlaw5\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\wlaw5\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model check

# Concatinate the two stirngs

In [ ]:
kana = "one two thirty-four"
kanji = "1 2 34"

"one two thirty-four" + " [SEP] " + "1 2 34"
kana + " [SEP] " + kanji

"one two thirty-four [SEP] 1 2 34"
3 = [SEP]

In [6]:
train_split_df['comibned']= train_split_df.text + "[SEP]" + train_split_df.kana
train_split_df.comibned

0       ホッケーにはデンジャラスプレーの反則があるので、膝より上にボールを浮かすことは基本的に反則に...
1       また行きたい、そんな気持ちにさせてくれるお店です。[SEP]またこうきたい、そんなきじちにさ...
2       手に持った特殊な刃物を使ったアクロバティックな体術や、揚羽と薄羽同様にクナイや忍具を使って攻...
3       3年次にはトータルオフェンスで2,892ヤードを獲得し、これは大学記録となった。[SEP]3...
4                             葬儀の最中ですよ![SEP]そうぎのさいちゅうですよ!
                              ...                        
7022                「さくらちゃん」と呼ばれている。[SEP]「さくらちゃん」とこばれている。
7023    シェルマルケ氏は新憲法などをめぐり、アハメド暫定大統領と対立していた。[SEP]シェルマルケ...
7024    自らがオウム真理教ではない別のカルト団体に12年間所属していた経験をもとに,こう語りました。...
7025    紅い髪と同じ色の瞳という容姿に加え、眉と鎖骨の下の部分に二つずつ紅い球体が埋まっているという...
7026    “人生,棒に振らないように”するためにはカルト団体からの速やかな離脱が必要だと感じるのは記者...
Name: comibned, Length: 7027, dtype: object

# Tokenize

convert everything into appropriate format for tenosrflow

In [21]:
inputs = tokenizer(train_split_df.comibned.values.tolist(),max_length=512,truncation=True,padding='max_length',return_tensors='np')
inputs

{'input_ids': array([[    2, 24543, 24893, ...,     0,     0,     0],
       [    2,   106,    77, ...,     0,     0,     0],
       [    2,   319,     7, ...,     0,     0,     0],
       ...,
       [    2,    98,    60, ...,     0,     0,     0],
       [    2,  4028,    21, ...,     0,     0,     0],
       [    2,  2203,    53, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

# save the kanji text before we mask, on a seprate varibal

In [22]:
inputs['labels'] = inputs['input_ids']


# MASK kanji text

In [ ]:
the apple [sep] fell down on the chari
1  2   64   3    7    234  34  75  12


the [sep][MASK] 
1   3   37   0   0 0   0   0  0 0

In [27]:
### inp_ids = []
inp_ids = []
idx = 0
for inp in inputs.input_ids:
    
    inp = np.array(inp)
    
    locatio_of_sep = np.where(inp == 3)[0][0]
    
    
    actual_tokens = list(set(range(locatio_of_sep,inp.shape[0])) - 
                         set(np.where((inp == 3) 
                            | (inp == 0))[0].tolist()))
    #We need to select 15% random tokens from the given list
    num_of_token_to_mask = int(len(actual_tokens)*0.50)
    token_to_mask = np.random.choice(np.array(actual_tokens), 
                                     size=num_of_token_to_mask, 
                                     replace=False).tolist()
    #Now we have the indices where we need to mask the tokens
    inp[token_to_mask] = 103
    inp_ids.append(inp)
    idx += 1
#inp_ids = tf.convert_to_tensor(inp_ids)
inputs['input_ids'] = np.array(inp_ids)


In [28]:
import tensorflow as tf

In [32]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
history = model.fit([inputs.input_ids,inputs.attention_mask],inputs.labels,verbose=1,batch_size=8,epochs=10)

Epoch 1/10
879/879 [==============================] - 14859s 17s/step - loss: 0.0514
Epoch 2/10
879/879 [==============================] - 15267s 17s/step - loss: 0.0191
Epoch 3/10
879/879 [==============================] - 14968s 17s/step - loss: 0.0126
Epoch 4/10
879/879 [==============================] - 14866s 17s/step - loss: 0.0112
Epoch 5/10
879/879 [==============================] - 14990s 17s/step - loss: 0.0086
Epoch 6/10
879/879 [==============================] - 14660s 17s/step - loss: 0.0085
Epoch 7/10
879/879 [==============================] - 14769s 17s/step - loss: 0.0073
Epoch 8/10
879/879 [==============================] - 14760s 17s/step - loss: 0.0059
Epoch 9/10
879/879 [==============================] - 14730s 17s/step - loss: 0.0055
Epoch 10/10
879/879 [==============================] - 14691s 17s/step - loss: 0.0062


In [33]:
model.save_pretrained("kana_to_kanji")
tokenizer.save_pretrained("kana_to_kanji")

('kana_to_kanji\\tokenizer_config.json',
 'kana_to_kanji\\special_tokens_map.json',
 'kana_to_kanji\\vocab.txt',
 'kana_to_kanji\\added_tokens.json')

# run it, convert 

In [34]:
from transformers import BertTokenizer, TFBertForMaskedLM

tokenizer = BertTokenizer.from_pretrained("kana_to_kanji")
model = TFBertForMaskedLM.from_pretrained("kana_to_kanji")

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at kana_to_kanji.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [ ]:
testColumn + '[SEP]' +   "[MASK]"*150

In [40]:
sentance = "自らがオウム真理教ではない別のカルト団体に12年間所属していた経験をもとに [SEP] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]"
inp = tokenizer(sentance,return_tensors='np')
mask_loc = np.where(inp.input_ids[0] == 103)[0].tolist()
out = model(inp).logits[0]
predicted_tokens = np.argmax(out[mask_loc],axis=1).tolist()
tokenizer.decode(predicted_tokens)

'[CLS] 自 らがオウム 真 理 教 ではない 別 のカルト 団 体 に12 年 間 所 属 していた 経 [UNK] をもとに [SEP] じらしょじいる 。 [SEP]'

In [42]:
test_split_df['masked_text'] = test_split_df.text + '[SEP]' +   "[MASK]"*150
test_split_df['masked_text']

0      これに不快感を示す住民はいましたが,現在,表立って反対や抗議の声を挙げている住民はいないよう...
1      幸福の科学側からは,特にどうしてほしいという要望はいただいていません。[SEP][MASK]...
2      星取り参加は当然とされ,不参加は白眼視される。[SEP][MASK][MASK][MASK]...
3      室長の対応には終始誠実さが感じられた。[SEP][MASK][MASK][MASK][MAS...
4      多くの女性が生理のことで悩んでいます。[SEP][MASK][MASK][MASK][MAS...
                             ...                        
538    プラスミドを改変してベクターとして利用する。[SEP][MASK][MASK][MASK][...
539    駐車場は狭いので混みあう時間帯は注意ですが、事前に電話をして到着時間を伝えておけば良いですね...
540    散髪のやり方次第で頭脳は発達すると考えて、どの店の散髪がよいか理髪店を絶えず替えていた。[S...
541    見上げる程の大きさの弧形が地表に露出しているが、これですら全体の一部でしかなく、全体は地殻を...
542    また、顕彰当時現役の選手も存在したが、いずれも既に全女を退団していた。[SEP][MASK]...
Name: masked_text, Length: 543, dtype: object

In [57]:
converted_sentance_list = []
for sentance in tqdm(test_split_df['masked_text'].values.tolist()[:20]):
    #sentance = "自らがオウム真理教ではない別のカルト団体に12年間所属していた経験をもとに [SEP] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]"
    inp = tokenizer(sentance,return_tensors='np')
    mask_loc = np.where(inp.input_ids[0] == 103)[0].tolist()
    out = model(inp).logits[0]
    predicted_tokens = np.argmax(out[mask_loc],axis=1).tolist()
    converted_sentance = tokenizer.decode(predicted_tokens)
    converted_sentance_list.append(converted_sentance.split('[SEP]')[0].strip('[CLS]').replace(" ",""))

converted_sentance_list

C:\Users\wlaw5\AppData\Local\Temp\ipykernel_332824\492050940.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sentance in tqdm(test_split_df['masked_text'].values.tolist()[:20]):


  0%|          | 0/20 [00:00<?, ?it/s]

['これに不快感を示す住民はいましたが,現在,表立って反対や抗議の声を挙げている住民はいないようです。',
 '幸福の科学側からは,特にどうしてほしいという要望はいただいていません。',
 '星取り参加は当然とされ,不参加は白眼視される。',
 '室長の対応には終始誠実さが感じられた。',
 '多くの女性が生理のことで悩んでいます。',
 '先生の理想は限りなく高い。',
 'それは兎も角,私も明日の社説を楽しみにしております。',
 'そうだったらいいなあとは思いますが,日本学術会議の会長談話について“当会では,標記の件について,以下のように考えます。”',
 '教団にとっては存続が厳しくなると思う。',
 'しかし強制していなくても問題です',
 '民族派のみなさんにとって[UNK]下はいちばん大切な方ですから,その霊言について“あり得ない”という前提でお話をされる。',
 '新しい産業構造を作らなければいけない。',
 '心がないんだ。',
 'そのモサは今何をしているか。',
 '本音としてはこの火に油を注ぎたいけれど。',
 '25日も楽しみにされてください。',
 '[UNK]~,わが署に来たのなら教[UNK]~。',
 'どの本もツッコミどころが山ほどあり,会場内が爆笑の連続でした。',
 '社会への恨みと,破滅させたいという恐ろしいほどの煩悩。',
 'とんでもない。']

In [ ]:
なんだよなんだよぉ~,わが署に来たのなら教えてくれよぉ~。

In [58]:
test_split_df.kana[:20].values.tolist()

['これにふかいかんをじすじゅうみんはいましたが,げんざい,ひょうりつってはんたいやこうぎのせいをきょげているじゅうみんはいないようです。',
 'こうふくのかがくそくからは,とくにどうしてほしいというようぼうはいただいていません。',
 'せいしゅりさんかはとうぜんとされ,ふさんかははくがんしされる。',
 'しつちょうのたいおうにはしゅうしせいじつさがかんじられた。',
 'たくのじょせいがせいりのことでのうんでいます。',
 'せんせいのりそうはげんりなくこうい。',
 'それはともかく,しもめいにちのしゃせつをがくしみにしております。',
 'そうだったらいいなあとはしいますが,にちほんがくじゅつかいぎのかいちょうだんわについて“とうかいでは,ひょうきのけんについて,いかのようにこうえます。”',
 'きょうだんにとってはそんぞくがげんしくなるとしう。',
 'しかしきょうせいしていなくてももんだいです',
 'みんぞくはのみなさんにとってへいかはいちばんだいせつなほうですから,そのれいげんについて“ありとくない”というぜんていでおわをされる。',
 'しんしいさんぎょうこうぞうをさくらなければいけない。',
 'しんがないんだ。',
 'そのモサはこんかをしているか。',
 'ほんおんとしてはこのかにゆをちゅうぎたいけれど。',
 '25にちもがくしみにされてください。',
 'なんだよなんだよぉ~,わがしょにらいたのならきょうえてくれよぉ~。',
 'どのほんもツッコミどころがさんほどあり,かいじょうないがばくしょうのれんぞくでした。',
 'しゃかいへのこんみと,はめつさせたいというきょうろしいほどのはんのう。',
 'とんでもない。']

In [61]:
test_split_df.text[:20].values.tolist()

['これに不快感を示す住民はいましたが,現在,表立って反対や抗議の声を挙げている住民はいないようです。',
 '幸福の科学側からは,特にどうしてほしいという要望はいただいていません。',
 '星取り参加は当然とされ,不参加は白眼視される。',
 '室長の対応には終始誠実さが感じられた。',
 '多くの女性が生理のことで悩んでいます。',
 '先生の理想は限りなく高い。',
 'それは兎も角,私も明日の社説を楽しみにしております。',
 'そうだったらいいなあとは思いますが,日本学術会議の会長談話について“当会では,標記の件について,以下のように考えます。”',
 '教団にとっては存続が厳しくなると思う。',
 'しかし強制していなくても問題です',
 '民族派のみなさんにとって陛下はいちばん大切な方ですから,その霊言について“あり得ない”という前提でお話をされる。',
 '新しい産業構造を作らなければいけない。',
 '心がないんだ。',
 'そのモサは今何をしているか。',
 '本音としてはこの火に油を注ぎたいけれど。',
 '25日も楽しみにされてください。',
 'なんだよなんだよぉ~,わが署に来たのなら教えてくれよぉ~。',
 'どの本もツッコミどころが山ほどあり,会場内が爆笑の連続でした。',
 '社会への恨みと,破滅させたいという恐ろしいほどの煩悩。',
 'とんでもない。']